install.packages("shiny", repos = "http://cran.us.r-project.org")

install.packages("CANSIM2R", repos = "http://cran.us.r-project.org")

install.packages("plotly", repos = "http://cran.us.r-project.org")

In [14]:
library(shiny)

In [ ]:
runExample("01_hello")


Listening on http://127.0.0.1:3979


In [12]:
library(CANSIM2R)
library(tidyverse)

In [2]:
gdp_raw <- getCANSIM(36100434, raw=TRUE)

In [5]:
print(unique(gdp_raw$Seasonal.adjustment))
print(unique(gdp_raw$Prices))
print(unique(gdp_raw$GEO))

[1] "Seasonally adjusted at annual rates" "Trading-day adjusted"               
[1] "Chained (2007) dollars" "2007 constant prices"  
[1] "Canada"


In [ ]:
gdp <- gdp_raw %>% 
  filter(Seasonal.adjustment %in% "Seasonally adjusted at annual rates")%>%
  select("time"="X.U.FEFF.REF_DATE",
         "naics"="North.American.Industry.Classification.System..NAICS.",
         "value"="VALUE", 
         "prices"="Prices") 

gdp %>% as_tibble() %>% print()

In [9]:
gdp$time1<-as.Date(paste((gdp$time), "-01", sep=""))
gdp$time1<-as.Date(gdp$time1)
gdp %>% as_tibble() %>% print()

# A tibble: 141,414 x 5
   time    naics                          value prices               time1     
   <chr>   <chr>                          <int> <chr>                <date>    
 1 1997-01 All industries               1047885 Chained (2007) doll~ 1997-01-01
 2 1997-01 Goods-producing industries    364724 Chained (2007) doll~ 1997-01-01
 3 1997-01 Service-producing industries  683158 Chained (2007) doll~ 1997-01-01
 4 1997-01 Business sector industries    852880 Chained (2007) doll~ 1997-01-01
 5 1997-01 Business sector, goods        362385 Chained (2007) doll~ 1997-01-01
 6 1997-01 Business sector, services     491968 Chained (2007) doll~ 1997-01-01
 7 1997-01 Non-business sector industr~  196274 Chained (2007) doll~ 1997-01-01
 8 1997-01 Non-business sector, goods      2311 Chained (2007) doll~ 1997-01-01
 9 1997-01 Non-business sector, servic~  193945 Chained (2007) doll~ 1997-01-01
10 1997-01 Industrial production         285617 Chained (2007) doll~ 1997-01-01
# ... with 141,4

In [15]:
# Define UI for app that analyzes the GDP at the national level for all NAICS industries ----

ui <- fluidPage(
  
  # App title ----
  titlePanel("GDP Analysis"),
  
  # Sidebar layout with input and output definitions ----
  sidebarLayout(
    
    # Sidebar panel for inputs ----
    sidebarPanel(
      
      # Input: selectInput allows users to choose a given industry ----
      selectInput(inputId = "industry",
                  label = "Select an industry:",
                  choices = unique(gdp$naics)), 
      
      # Input: radioButtons allows users to choose a price measure ----
      radioButtons(inputId = "prices",
                   label = "Select price measure:",
                   choices = unique(gdp$prices))
      
    ),
    
    # Main panel for displaying outputs ----
    mainPanel(
      
      # Output: Plot ----
      plotOutput(outputId = "plot2", height = "800px", width = "700px")
      
      
    )
  )
)

# Define server logic required to draw a histogram ----
server <- function(input, output) {
  

  # This expression will generate a plot that is wrapped in a call
  # to renderPlot to indicate that:
  #
  # 1. It is "reactive" and therefore should be automatically
  #    re-executed when inputs from the ui change
  #
  # 2. Its output type is a plot
  
  
  output$plot2 <- renderPlot({
    
    dat <- gdp %>%
      filter(naics %in% input$industry ) %>%
      filter(prices %in% input$prices)
    
    ggplot(dat, aes(x= time1 , y= value, group=1))+
      geom_line()+
      theme(axis.text.x = element_text(size=11, angle=90))+
      scale_x_date(date_labels = "%b %y", date_breaks = "1 year")

  })
  
}

# To run the app:

# shinyApp(ui=ui, server=server)

In [ ]:
shinyApp(ui=ui, server=server)


Listening on http://127.0.0.1:6226
